<a href="https://colab.research.google.com/github/arzhrd/Basketball-Player-Detail-Using-Computer-Vision/blob/main/Basketball_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Configure API Keys and check for GPU
import os
from google.colab import userdata
from pathlib import Path
import torch

# Load API keys from Colab secrets
try:
    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
    os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")
    print("API keys loaded successfully.")
except Exception as e:
    print("Could not load API keys. Please set them up in Colab Secrets (🔑).")
    print("Required secrets: 'HF_TOKEN' and 'ROBOFLOW_API_KEY'")

# Check for GPU
!nvidia-smi

# Set home directory
HOME = Path.cwd()
print("HOME:", HOME)

# Set ONNX provider to use GPU
os.environ["ONNXRUNTIME_EXECUTION_PROVIDERS"] = "[CUDAExecutionProvider]"

# 2. Install SAM2 (Segment Anything Model 2) for tracking
!git clone https://github.com/Gy920/segment-anything-2-real-time.git
%cd {HOME}/segment-anything-2-real-time
!pip install -e . -q
!python setup.py build_ext --inplace
!(cd checkpoints && bash download_ckpts.sh)
%cd {HOME}

# 3. Install all other required Python packages
!pip install -q gdown inference-gpu supervision transformers num2words
!pip install -q git+https://github.com/roboflow/sports.git@feat/basketball
!pip install -q flash-attn --no-build-isolation

print("\n✅ All installations are complete.")

In [ ]:
# 1. Download sample videos and fonts
SOURCE_VIDEO_DIRECTORY = HOME / "source"
!gdown -q https://drive.google.com/drive/folders/1eDJYqQ77Fytz15tKGdJCMeYSgmoQ-2-H -O {SOURCE_VIDEO_DIRECTORY} --folder
!gdown -q https://drive.google.com/drive/folders/1RBjpI5Xleb58lujeusxH0W5zYMMA4ytO -O {HOME / "fonts"} --folder
print("Sample videos and fonts downloaded.")

# 2. Define the source video path you want to process
# You can change the filename to process a different clip from the `source` directory
SOURCE_VIDEO_PATH = SOURCE_VIDEO_DIRECTORY / "boston-celtics-new-york-knicks-game-1-q1-04.28-04.20.mp4"

# 3. Define Team Rosters and Colors
TEAM_ROSTERS = {
  "New York Knicks": {
    "55": "Hukporti", "1": "Payne", "0": "Wright", "11": "Brunson", "3": "Hart",
    "32": "Towns", "44": "Shamet", "25": "Bridges", "2": "McBride",
    "23": "Robinson", "8": "Anunoby", "4": "Dadiet", "5": "Achiuwa", "13": "Kolek"
  },
  "Boston Celtics": {
    "42": "Horford", "55": "Scheierman", "9": "White", "20": "Davison",
    "7": "Brown", "0": "Tatum", "27": "Walsh", "4": "Holiday", "8": "Porzingis",
    "40": "Kornet", "88": "Queta", "11": "Pritchard", "30": "Hauser",
    "12": "Craig", "26": "Tillman"
  }
}

TEAM_COLORS = {
    "New York Knicks": "#006BB6",
    "Boston Celtics": "#007A33"
}